# 传感器介绍

## 1. 激光雷达

激光雷达，英文全称为Light Detection And Ranging，简称LiDAR，即光探测与测量，是一种集激光、全球定位系统（GPS）和IMU（InertialMeasurement Unit，惯性测量设备）三种技术于一身的系统，用于获得数据并生成精确的DEM（数字高程模型）。这三种技术的结合，可以高度准确地定位激光束打在物体上的光斑，测距精度可达厘米级，激光雷达最大的优势就是“精准”和“快速、高效作业”。它是一种用于精确获得3D位置信息的传感器，其在机器中的作用相当于人类的眼睛，能够确定物体的位置、大小、外部形貌甚至材质。[参考](https://cloud.tencent.com/developer/news/387139)

### 原理：

[激光雷达LIDAR工作原理](https://www.sohu.com/a/334652375_326515)

### 分类：

* 机械vs固态：旋转/固定
* [三角测距vs飞行时间](https://blog.csdn.net/weixin_42229404/article/details/83825448)：从距离测定的原理上划分

### 关键参数

* 测量距离范围
* 测量角度范围（横向、纵向）
* 角度分辨率（横向、纵向）
* 距离测量精度

### [干扰因素](https://laser.ofweek.com/2020-06/ART-240001-11000-30442590.html)

### 示例：
* [三角测距原理的开源硬件](https://github.com/iliasam/OpenSimpleLidar)
* [飞行时间原理的开源硬件](https://github.com/iliasam/OpenTOFLidar)

关注硬件结构、测量效果、ROS驱动等方面。

* [velodyne ROS驱动](https://github.com/ros-drivers/velodyne)
* [rslidar ROS驱动](https://github.com/RoboSense-LiDAR/ros_rslidar)

激光雷达数据示例

## 2. IMU

IMU，即惯性测量单元，是一种常见的传感器。应用于移动机器人的IMU一般输出两类测量值：

* 绕空间坐标轴x、y、z旋转的角速度
* 沿空间坐标轴x、y、z方向的加速度

因此，IMU有时可以看作是“加速度计”和“陀螺仪”的结合体。IMU数据示例

加速度计和陀螺仪的误差可以分为：确定性误差（系统误差），随机误差。

确定性误差可以事先标定确定，包括：bias，scale等；随机误差通常假设噪声服从高斯分布，包括：高斯白噪声，bias随机游走等

下面我们重点讨论无法方便地事先确定的随机误差：

### IMU随机误差

#### a. 高斯白噪声：

这一项随机误差的名字可能有很多，例如“高斯白噪声”、“随机行走误差（random walk）”、“Noise density”等，表达的都是这项内容。以陀螺仪为例，单位为dps/rt(Hz)，或者deg/rt(h)。

陀螺仪测量的角速度，加速度计测量的加速度的噪声是高斯白噪声。因此这一项误差表示的物理意义就是单位时间内角度不确定性（标准差）、速度不确定性（标准差）的增量。

例如，如果陀螺仪的的随机行走误差是20deg/rt(h)，那意味着在一个小时后，积分得到的角度的误差的标准差是20deg。


以下表格为IMU随机误差的单位：

Parameter | YAML element | Symbol | Units
--- | --- | --- | ---
Gyroscope "white noise" | `gyr_n` | <img src="https://latex.codecogs.com/svg.latex?{%5Csigma_g}"> | <img src="https://latex.codecogs.com/svg.latex?{%5Cfrac%7Brad%7D%7Bs%7D%5Cfrac%7B1%7D%7B%5Csqrt%7BHz%7D%7D}">
Accelerometer "white noise" | `acc_n` | <img src="https://latex.codecogs.com/svg.latex?{%5Csigma_a}"> | <img src="https://latex.codecogs.com/svg.latex?{%5Cfrac%7Bm%7D%7Bs^2%7D%5Cfrac%7B1%7D%7B%5Csqrt%7BHz%7D%7D}">
Gyroscope "bias Instability" | `gyr_w` | <img src="https://latex.codecogs.com/svg.latex?{%5Csigma_b_g}"> | <img src="http://latex.codecogs.com/svg.latex?\frac{rad}{s}&space;\sqrt{Hz}" title="\frac{rad}{s} \sqrt{Hz}" />
Accelerometer "bias Instability" | `acc_w` | <img src="https://latex.codecogs.com/svg.latex?{%5Csigma_b_a}"> | <img src="http://latex.codecogs.com/svg.latex?\frac{m}{s^2}&space;\sqrt{Hz}" title="\frac{m}{s^2} \sqrt{Hz}" />


#### b. 零偏不稳定性（bias instability）：

理论上讲，当IMU完全不受激励时，输出应该为零，但现实往往不是这样。一般我们将传感器所受激励为零时的输出读数成为传感器的“零偏（bias）”。IMU的零偏属于系统误差，可以通过某些操作在事先进行标定。但IMU在使用过程中，会出现零偏慢慢变化的情况。直观上理解，零偏不稳定性误差会使陀螺仪和加速度计的零偏随时间慢慢变化，逐渐偏离开机时校准的零偏误差；同时还会造成一部分随机行走误差的效果。可以简要理解为在一段时间内零偏的变化量，其他名称可能还有“零偏随机游走（bias random walk）”、“零偏稳定性（bias stability）”等。

### 如何获得IMU随机误差参数

根据瑞士苏黎世理工学院的kalibr标定工具库中的介绍：[IMU-Noise-Model](https://github.com/ethz-asl/kalibr/wiki/IMU-Noise-Model)，IMU随机误差参数可以由以下方式获得：

* 1. 厂家提供的参数表

* 2. Allan分析法：

Allan标准差分析法是通过对一段时间内的已采集IMU数据估计IMU随机误差参数的一种数值计算方法。[简要介绍](https://zhuanlan.zhihu.com/p/71202815) 

网上有很多根据这一方法编写的程序，可以用来处理IMU数据从而得到随机误差参数，例如香港科技大学研究人员提供的工具[imu_utils](https://github.com/gaowenliang/imu_utils)：按照README介绍的流程，即可得到输出的acc_n acc_w gyr_n gyr_w等。

### 讨论：随机误差参数的离散化

根据[IMU-Noise-Model](https://github.com/ethz-asl/kalibr/wiki/IMU-Noise-Model)和[讨论](https://blog.csdn.net/liu2015302026/article/details/107792651)中的内容，参数表和Allan标准差分析法得出的都是连续噪声模型的误差参数，在具体应用时，需要使用离散化噪声模型的误差参数，也就是根据所使用IMU的输出频率进行修正。如以上两个链接所示。

IMU随机误差参数的具体应用场景例如：
* [kalibr](https://github.com/ethz-asl/kalibr/wiki/yaml-formats)
* [VINS](https://github.com/HKUST-Aerial-Robotics/VINS-Fusion/blob/master/config/mynteye/mynteye_stereo_imu_config.yaml)

但就如[IMU-Noise-Model](https://github.com/ethz-asl/kalibr/wiki/IMU-Noise-Model)最后所说，噪声模型推导出的随机误差参数都是在传感器静止和恒定温度下得出的，在IMU运动过程中，随机误差会更大，因此具体使用这些随机误差参数时，建议将参数扩大10倍以上使用

## 3. 相机



相机可以产生连续的图像，每张图像由像素组成，每个像素承载一定信息，如颜色等。三维世界中的一个物体反射或发出的光线,穿过相机光心后,投影在相机的成像平面上。相机的感光器件接收到光线后,产生了测量值,就得到了像素,形成了我们见到的照片。

### 相机模型

相机将三维世界中的坐标点(单位为米)映射到二维图像平面(单位为像素)的过程能够用一个几何模型进行描述。这个模型有很多种,其中最简单的称为针孔模型。针孔模型是很常用,而且有效的模型,它描述了一束光线通过针孔之后,在针孔背面投影成像的关系。同时,由于相机镜头上的透镜的存在,会使得光线投影到成像平面的过程中会产生畸变。因此,我们使用针孔和畸变两个模型来描述整个投影过程。

(x_u,x_v)=f(g(x_c,y_c,z_c))            [1]

我们使用上式[1]来帮助理解这个过程：假如x_c，y_c，z_c是三维世界中的坐标点，函数g()处理畸变，函数f()处理三维世界到相机二维成像平面的投影关系，最终得到相机成像平面上的点的坐标x_u，x_v。这样一来，获取了相机图像的算法就能根据（x_u，x_v)来反推某点在三维世界中的位置了。而所谓的相机“标定”过程就是通过某些操作来确定函数g()和f()中的一些参数的过程。

### 3.1 针孔相机模型

针孔相机模型可以看做是函数f()的一种形式，具有fx、fy、cx、cy等参数。

![针孔相机模型](img/pinhole.png)

![相机内参](img/Camera_Intrinsics.png)

我们把中间的量组成的矩阵称为相机的内参数矩阵(Camera Intrinsics)K。通常认为,相机的内参在出厂之后是固定的,不会在使用过程中发生变化。有的相机生产厂商会告诉你相机的内参,而有时需要你自己确定相机的内参,也就是所谓的标定。

针孔相机模型（pinhole）是最简单的一种相机模型，其他诸如MEI模型、全向模型等可以看作是函数f()的其他实现形式。

### 3.2 畸变模型

为了获得好的成像效果,我们在相机的前方加了透镜。透镜的加入对成像过程中光线的传播会产生新的影响: 一是透镜自身的形状对光线传播的影响,二是在机械组装过程中,透镜和成像平面不可能完全平行,这也会使得光线穿过透镜投影到成像面时的位置发生变化。

由透镜形状引起的畸变称之为径向畸变。在针孔模型中,一条直线投影到像素平面上还是一条直线。可是,在实际拍摄的照片中,摄像机的透镜往往使得真实环境中的一条直线在图片中变成了曲线。越靠近图像的边缘,这种现象越明显。由于实际加工制作的透镜往往是中心对称的,这使得不规则的畸变通常径向对称。它们主要分为两大类,桶形畸变和枕形畸变,如图所示：

![径向畸变](img/distortion.png)

除了透镜的形状会引入径向畸变外,在相机的组装过程中由于不能使得透镜和成像面严格平行也会引入切向畸变。

![切向畸变](img/distortion2.png)

根据径向畸变和切向畸变的模型修复失真图像，就是函数g()的一种实现过程，具有k1、k2、k3、p1、p2等参数。

### 3.3 双目相机模型

针孔相机模型描述了单个相机的成像模型。然而,仅根据一个像素,我们是无法确定这个空间点的具体位置的。这是因为,从相机光心到归一化平面连线上的所有点,都可以投影至该像素上。只有当 P 的深度确定时(比如通过双目或 RGB-D 相机),我们才能确切地知道它的空间位置。测量像素距离(或深度)的方式有很多种,像人眼就可以根据左右眼看到的景物差异(或称视差)来判断物体与我们的距离。双目相机的原理亦是如此。

![双目测距](img/stereo.png)

在上图中，假设三维世界中的物体P在左目相机成像平面上的像素位置PL，和在右目相机成像平面上的像素位置PR是已知的，即图中u_L和u_R已知，f焦距和b相机左右目间距是相机固有参数，那么可以求出P的距离z。

### 3.4 RGB-D相机模型

相比于双目相机通过视差计算深度的方式,RGB-D 相机的做法更为“主动”一些,它能够主动测量每个像素的深度。目前的 RGB-D 相机按原理可分为两大类:

* 1. 通过红外结构光 (Structured Light)来测量像素距离的。例子有 Kinect 1 代、 Project Tango 1 代、Intel RealSense 等;
* 2. 通过飞行时间法(Time-of-flight, ToF)原理测量像素距离的。例子有 Kinect 2 代和一些现有的 ToF 传感器等。

无论是结构光还是 ToF,RGB-D 相机都需要向探测目标发射一束光线(通常是红外光)。在结构光原理中,相机根据返回的结构光图案,计算物体离自身的距离。而在 ToF中,相机向目标发射脉冲光,然后根据发送到返回之间的光束飞行时间,确定物体离自身的距离。ToF 原理和激光传感器十分相似,不过激光是通过逐点扫描来获取距离,而 ToF相机则可以获得整个图像的像素深度,这也正是 RGB-D 相机的特点。所以,如果你把一个 RGB-D 相机拆开,通常会发现除了普通的摄像头之外,至少会有一个发射器和一个接收器。

## 4. 传感器参数标定

标定过程

kalibr相机-imu外参标定

[小觅相机内参标定](https://mynt-eye-s-sdk.readthedocs.io/zh_CN/latest/src/tools/calibration_tool.html)

标定后的参数去哪，VINS